In [3]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define function to retrieve data for a given file
def get_grs_data(file_name):
    # Define base URL for PDS API
    base_url = 'https://pds-gamma.jpl.nasa.gov/api/v1/data/lpgrs/'
    # Define URL for file of interest
    file_url = base_url + file_name
    # Send GET request to retrieve file
    response = requests.get(file_url)
    # Load binary data from response content
    data = np.frombuffer(response.content, dtype='>f4')
    # Reshape data into 128 x 256 array
    data = data.reshape((128, 256))
    return data

# Define function to sum data within given latitude and longitude ranges
def sum_grs_data(data, lat_range, lon_range):
    # Extract latitude and longitude arrays from data
    lat = np.arange(-64.75, 65, 1.5)
    lon = np.arange(-180, 180, 1.40625)
    # Find indices for latitude and longitude ranges
    lat_min, lat_max = lat_range
    lon_min, lon_max = lon_range
    lat_idx = np.where((lat >= lat_min) & (lat <= lat_max))[0]
    lon_idx = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    # Sum data within specified ranges
    sum_data = np.sum(data[lat_idx, :][:, lon_idx], axis=(0,1))
    return sum_data

# Define function to plot spectra
def plot_spectra(spectra, labels):
    fig, ax = plt.subplots()
    for i in range(len(spectra)):
        ax.plot(spectra[i]/sum(spectra[i]), label=labels[i])
    ax.set_xlabel('Channel')
    ax.set_ylabel('Normalized Count Rate')
    ax.set_title('Summed Spectra')
    ax.legend()
    plt.show()

# Set altitude ranges
alt_100km = (90, 110)
alt_30km = (27, 33)

# Set latitude and longitude ranges
ranges_100km = [
    ((0, 30), (-45, 0)),
    ((-15, 15), (20, 40)),
    ((20, 45), (90, 120))
]

ranges_30km = [
    ((0, 30), (-45, 0)),
    ((-15, 15), (20, 40)),
    ((20, 45), (90, 120))
]

# Initialize empty lists for spectra and labels
spectra_100km = []
labels_100km = []
spectra_30km = []
labels_30km = []

# Loop through all data files and sum data within specified ranges
for year in range(1998, 1999):
    for doy in range(1, 367):
        # Construct file name
        file_name = f'{year}_{doy:03}_grs.dat'
        # Retrieve data for file
        data = get_grs_data(file_name)
        # Sum data for 100 km altitude within specified ranges
        if alt_100km[0] <= year < alt_100km[1]:
            for i, (lat_range, lon_range) in enumerate(ranges_100km):
                label = f'{year}_{doy:03}_100km_{i+1}'
                spectra_100km.append(sum_grs_data(data, lat_range, lon_range))


ValueError: buffer size must be a multiple of element size